# Exploratory Data Analysis

This notebook will cover EDA on the dataset 'vehicles_us.csv'

## Imports

In [13]:
import pandas as pd
import numpy as np
import plotly.express as px
import notebook_utils.notebook_auxiliaries as aux

## Load

In [14]:
data = pd.read_csv('../data/vehicles_us.csv')
data.info()
data.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
9482,17995,2009.0,ford f250,excellent,8.0,diesel,NaN,automatic,truck,custom,1.0,2019-02-06,19
10997,21600,2016.0,chevrolet camaro lt coupe 2d,good,NaN,gas,19579.0,other,coupe,grey,NaN,2018-05-20,29
21937,8495,2009.0,honda pilot,excellent,6.0,gas,121107.0,automatic,SUV,silver,1.0,2019-02-22,57
149,13900,2016.0,dodge grand caravan,excellent,6.0,gas,62711.0,automatic,van,black,NaN,2019-04-11,37
21762,6800,2010.0,nissan rogue,good,4.0,gas,108323.0,automatic,hatchback,NaN,1.0,2018-11-28,11


File seems to have been read successfully and column names are ok.

## Data Cleaning

FixingIn this section I'll fix missing malues, deal with duplicates, and fit adequate data types.

### Missing values

In [15]:
aux.isna_report(data)

*** Missing/Null values report ***
----------------------------------
Single column analysis

3619 (7.0%) missing values in column: 'model_year'
5260 (10.2%) missing values in column: 'cylinders'
7892 (15.3%) missing values in column: 'odometer'
9267 (18.0%) missing values in column: 'paint_color'
25953 (50.4%) missing values in column: 'is_4wd'

Multiple column analysis (2)

363 (0.7%) concurrent missing values in: ('model_year', 'cylinders')
549 (1.1%) concurrent missing values in: ('model_year', 'odometer')
652 (1.3%) concurrent missing values in: ('model_year', 'paint_color')
1811 (3.5%) concurrent missing values in: ('model_year', 'is_4wd')
812 (1.6%) concurrent missing values in: ('cylinders', 'odometer')
950 (1.8%) concurrent missing values in: ('cylinders', 'paint_color')
2681 (5.2%) concurrent missing values in: ('cylinders', 'is_4wd')
1455 (2.8%) concurrent missing values in: ('odometer', 'paint_color')
4016 (7.8%) concurrent missing values in: ('odometer', 'is_4wd')
4637 (9.

* The column 'is_4wd' is 50% missing values, it'll be removed.
* The rows with missing values in 'model_year' amount to less than 10% of all the rows, they can be dropped.
* The rows with missing values in 'cylinders' amount to around 10% of all the rows, they can be dropped.
* The column 'paint_color' has over 17% missing values, they will be inputed with the most common color (the mode) for cars of the same model ('model' column).
* The column 'odometer' has around 15% missing values, they'll be inputed with the mean odometer readings for cars of the same condition and same model ('condition' & 'model' columns).
* Rows with multiple missing values are rare in the data set.

In [16]:
data = aux.fix_nas(data)

*** Fixing missing values ***
-----------------------------
Dropping column: 'is_4wd'...
Dropping rows with missing values from 'model_year' and 'cylinders'...
Inputting values for 'paint_color'...
Inputting values for 'odometer'...
Starting rows: 51525 Rows after: 43009 (8516 rows lost, 16.5%)
Starting columns: 13 Columns after: 12 (1 columns lost, 7.7%)
-----------------------------
+++ Missing values FIXED +++


Report on missing values, again.

In [17]:
aux.isna_report(data)

*** Missing/Null values report ***
----------------------------------
No missing values found.
-----------------------------------
+++ Report END +++


### Duplicates

In [18]:
data.duplicated().sum()

np.int64(0)

* There are no explicit, full-row duplicates.
* No variable or combination of variables seems to work as a unique identifier.
* There is no need to check for duplicates in individual columns or combinations of columns.

### Data types

In [19]:
data = aux.fix_data_types(data)

Fitting data type for column: 'model'
	cardinality: 100
	due to context as category, casting to 'category'
Fitting data type for column: 'condition'
	cardinality: 6
	due to context as category, casting to 'category'
Fitting data type for column: 'fuel'
	cardinality: 5
	due to context as category, casting to 'category'
Fitting data type for column: 'transmission'
	cardinality: 3
	due to context as category, casting to 'category'
Fitting data type for column: 'type'
	cardinality: 13
	due to context as category, casting to 'category'
Fitting data type for column: 'paint_color'
	cardinality: 12
	due to context as category, casting to 'category'
Fitting data type for column: 'price'
	min: 1	max: 375000
	requires negative values?: False
	requires decimal part?: False
	due to monetary context, casting to 'float32'
Fitting data type for column: 'model_year'
	min: 1908.0	max: 2019.0
	requires negative values?: False
	requires decimal part?: False
	due to context as year, casting to 'uint16'
Fit

Results of data type correction:

In [20]:
# reset index since we dropped some rows
data = data.reset_index(drop=True)
data.info()
data.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43009 entries, 0 to 43008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   price         43009 non-null  float32       
 1   model_year    43009 non-null  uint16        
 2   model         43009 non-null  category      
 3   condition     43009 non-null  category      
 4   cylinders     43009 non-null  uint8         
 5   fuel          43009 non-null  category      
 6   odometer      43009 non-null  float32       
 7   transmission  43009 non-null  category      
 8   type          43009 non-null  category      
 9   paint_color   43009 non-null  category      
 10  date_posted   43009 non-null  datetime64[ns]
 11  days_listed   43009 non-null  uint16        
dtypes: category(6), datetime64[ns](1), float32(2), uint16(2), uint8(1)
memory usage: 1.1 MB


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,date_posted,days_listed
26192,10995.0,2008,chevrolet silverado 1500,good,8,gas,168200.000000,automatic,truck,blue,2018-09-18,38
26320,13995.0,2013,honda pilot,good,6,gas,93714.000000,automatic,SUV,black,2019-01-14,7
27133,2500.0,2006,volkswagen passat,fair,4,gas,190000.000000,manual,sedan,grey,2018-06-28,17
21224,24988.0,2017,chevrolet silverado,like new,8,gas,64000.000000,automatic,truck,white,2018-06-14,69
40685,30990.0,2015,chevrolet silverado 3500hd,excellent,8,diesel,106274.109375,automatic,other,white,2018-12-11,18


### Data cleaning: Results

* I've removed an unrecoverable column.
* I've inputted sensible data to cover missing values.
* No duplicates were detected.
* Data types have been fitted that better represent the data. 

## Exporting the data

Now that I've visualized the clean data, I will export it to a file available for my streamlit app. An ancilliary YAML file will be created containing a dictionary structure matching each variable with it's most appropriate data type. These files will then be consumed by the online app so it doesn't have to repeat the data cleaning process.

In [21]:
aux.export_clean_data(data, '../data/clean_vehicles_us.csv', '../data/clean_vehicles_us_dtypes.yaml')

Exporting dataframe to file: '../data/clean_vehicles_us.csv'
	../data/clean_vehicles_us.csv created successfully.
Creating data types correction dictionary...
	column: 'price', is of type: 'float32'
	column: 'model_year', is of type: 'uint16'
	column: 'model', is of type: 'category'
	column: 'condition', is of type: 'category'
	column: 'cylinders', is of type: 'uint8'
	column: 'fuel', is of type: 'category'
	column: 'odometer', is of type: 'float32'
	column: 'transmission', is of type: 'category'
	column: 'type', is of type: 'category'
	column: 'paint_color', is of type: 'category'
	column: 'date_posted', is of type: 'datetime64[ns]'
	column: 'days_listed', is of type: 'uint16'
Datetime format for the only date column is '%Y-%m-%d'
Output dictionary preview:
{'columns': {'price': 'float32', 'model_year': 'uint16', 'model': 'category', 'condition': 'category', 'cylinders': 'uint8', 'fuel': 'category', 'odometer': 'float32', 'transmission': 'category', 'type': 'category', 'paint_color': 

## Exploration

Now that I have clean data, I want to visualize it to discover trends.

### Odometer histogram

In [22]:
# assuming the vehicles in the us have their odometers in miles
hist_odometer = px.histogram(data, x='odometer', 
                             labels={'odometer':'Odometer reading (total distance traveled) in millions of miles*'},
                             title='<b>How long a distance do our vehicles travel?</b>')
hist_odometer.update_traces(marker_color='teal')
hist_odometer.show()

It seems most vehicles travel between 0 and 200,000 miles, but the have outliers that travel almost 1 million miles.

### How does distance traveled affects price?

In [23]:
scatter_price_vs_odometer = px.scatter(data, x='odometer', y='price',
                                       labels={'odometer':'Odometer reading (total distance traveled) in millions of miles',
                                               'price':'Price in USD'},
                                       title='<b>Does heavy use depreciate vehicles?</b>')
scatter_price_vs_odometer.update_traces(marker_color='teal')
scatter_price_vs_odometer.show() 

The less used a vehicle is the higher its price.

### Donut chart

In [24]:
donut_data = data.groupby('fuel', observed=True)['cylinders'].count().reset_index().rename(columns={'cylinders':'unit_count'})
donut_fuel = px.pie(donut_data, values='unit_count', names='fuel', hole=0.4,
             title='<b>Which fuel types are more common among the vehicles we sell?</b>', 
             color_discrete_sequence=px.colors.sequential.Blugrn_r)
donut_fuel.show()


Because I wanted to make one.

# Conclusions

Dummy analysis tasks done. Time to publish to Streamlit.